# Data Extraction

## Step 1: Extracting Text

In [ ]:
# we will frist read the csv from file and start extracting text from the pages'

from text_functions import get_text, clean_text, read_from_csv
pages_records=read_from_csv('./page_labeled.csv')
pages_records

In [ ]:
pages_records['Text']=get_text(pages_records['representative_page'])
pages_records['Text']=pages_records['Text'].apply(lambda x: clean_text(x))

In [ ]:
pages_records

## Step 2: Extract Case Numbers

In [ ]:
from model_functions import prepare_data_for_model
collected_hash=read_from_csv("collected_hash.csv")
_, encoded_data = prepare_data_for_model(collected_hash)

In [ ]:
# here you provide a bounding box for the code to search on a each of the pages. 
from data_extraction_functions import extract_with_bbox
possible_case_numbers= extract_with_bbox([0,0,1000,400], encoded_data, pages_records)

In [ ]:
possible_case_numbers

## Step 3: Extracting Date, Location, and Names

In [ ]:
from data_extraction_functions import get_named_entity
dates= get_named_entity(pages_records, 'DATE')
organizations= get_named_entity(pages_records, 'ORG')
names= get_named_entity(pages_records, 'PERSON')

In [ ]:
names

# Data Organization

## Step 1: Setting extracted entities

In [ ]:
from file_organization_functions import set_entity_whole, set_entity_breakdown

pages_records=set_entity_whole(pages_records, 'Date', dates)
pages_records=set_entity_whole(pages_records, 'Name', names)
# pages_records=set_entity_whole(pages_records, 'Organizations', organizations)
pages_records=set_entity_whole(pages_records, 'CaseNumber', possible_case_numbers)
pages_records

In [ ]:
# after page level data is collected, set the proper data points that were extracted to each of the documents. 
from file_organization_functions import set_doc_data

doc_df= set_doc_data(pages_records)
doc_df.to_csv(open("./doc_data.csv", "w"))

In [ ]:
doc_df

## Step 2: highlighting matching names

In [ ]:
# After this stage, how you organize the files and what you chose to highlight will depend on your data and your interest. There are functions in file_class.py that will 
# allow you to highlight texts that match accross documents and group files based on your parameters.

In [ ]:
# you can use this function to get a first set of files that are connected based on case numbers
# you can then explore your data and add to your case objects as you find more connected files beyond just case numbers. 
from file_class import find_related_files, create_case_objects

connected_files= find_related_files(doc_df)

In [ ]:
connected_files